## 6.8 a

In [7]:
# 6.8a
from math import sqrt

ORIG = ([0,0], "ORIGIN")
S0  = ([-3,  3], "S0 ")
S1  = ([-1,  3], "S1 ")
S2  = ([ 1,  3], "S2 ")
S3  = ([ 3,  3], "S3 ")
S4  = ([-3,  1], "S4 ")
S5  = ([-1,  1], "S5 ")
S6  = ([ 1,  1], "S6 ")
S7  = ([ 3,  1], "S7 ")
S8  = ([-3, -1], "S8 ")
S9  = ([-1, -1], "S9 ")
S10 = ([ 1, -1], "S10")
S11 = ([ 3, -1], "S11")
S12 = ([-3, -3], "S12")
S13 = ([-1, -3], "S13")
S14 = ([ 1, -3], "S14")
S15 = ([ 3, -3], "S15")

def get_distance(start, end):
    """Put one Sx into start and a list of the rest of Sx's into end"""
    for end_point in end:
        distance = sqrt((start[0][0] - end_point[0][0])**2 + (start[0][1] - end_point[0][1])**2)
        print(f"Distance from {start[1]:3} to {end_point[1]:3} = {distance}")

get_distance(S0, [S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15])


Distance from S0  to S1  = 2.0
Distance from S0  to S2  = 4.0
Distance from S0  to S3  = 6.0
Distance from S0  to S4  = 2.0
Distance from S0  to S5  = 2.8284271247461903
Distance from S0  to S6  = 4.47213595499958
Distance from S0  to S7  = 6.324555320336759
Distance from S0  to S8  = 4.0
Distance from S0  to S9  = 4.47213595499958
Distance from S0  to S10 = 5.656854249492381
Distance from S0  to S11 = 7.211102550927978
Distance from S0  to S12 = 6.0
Distance from S0  to S13 = 6.324555320336759
Distance from S0  to S14 = 7.211102550927978
Distance from S0  to S15 = 8.48528137423857


In [8]:

get_distance(ORIG, [S0, S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15])


Distance from ORIGIN to S0  = 4.242640687119285
Distance from ORIGIN to S1  = 3.1622776601683795
Distance from ORIGIN to S2  = 3.1622776601683795
Distance from ORIGIN to S3  = 4.242640687119285
Distance from ORIGIN to S4  = 3.1622776601683795
Distance from ORIGIN to S5  = 1.4142135623730951
Distance from ORIGIN to S6  = 1.4142135623730951
Distance from ORIGIN to S7  = 3.1622776601683795
Distance from ORIGIN to S8  = 3.1622776601683795
Distance from ORIGIN to S9  = 1.4142135623730951
Distance from ORIGIN to S10 = 1.4142135623730951
Distance from ORIGIN to S11 = 3.1622776601683795
Distance from ORIGIN to S12 = 4.242640687119285
Distance from ORIGIN to S13 = 3.1622776601683795
Distance from ORIGIN to S14 = 3.1622776601683795
Distance from ORIGIN to S15 = 4.242640687119285


In [9]:
get_distance(S1, [S0, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15])

Distance from S1  to S0  = 2.0
Distance from S1  to S2  = 2.0
Distance from S1  to S3  = 4.0
Distance from S1  to S4  = 2.8284271247461903
Distance from S1  to S5  = 2.0
Distance from S1  to S6  = 2.8284271247461903
Distance from S1  to S7  = 4.47213595499958
Distance from S1  to S8  = 4.47213595499958
Distance from S1  to S9  = 4.0
Distance from S1  to S10 = 4.47213595499958
Distance from S1  to S11 = 5.656854249492381
Distance from S1  to S12 = 6.324555320336759
Distance from S1  to S13 = 6.0
Distance from S1  to S14 = 6.324555320336759
Distance from S1  to S15 = 7.211102550927978


In [10]:
get_distance(S5, [S0, S1, S2, S3, S4, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15])

Distance from S5  to S0  = 2.8284271247461903
Distance from S5  to S1  = 2.0
Distance from S5  to S2  = 2.8284271247461903
Distance from S5  to S3  = 4.47213595499958
Distance from S5  to S4  = 2.0
Distance from S5  to S6  = 2.0
Distance from S5  to S7  = 4.0
Distance from S5  to S8  = 2.8284271247461903
Distance from S5  to S9  = 2.0
Distance from S5  to S10 = 2.8284271247461903
Distance from S5  to S11 = 4.47213595499958
Distance from S5  to S12 = 4.47213595499958
Distance from S5  to S13 = 4.0
Distance from S5  to S14 = 4.47213595499958
Distance from S5  to S15 = 5.656854249492381
